In [25]:
import os
from nipype.interfaces import fsl
from nipype.interfaces.io import SelectFiles, DataSink
import nipype.pipeline.engine as pe
import nipype.interfaces.utility as util
import nipype.algorithms.modelgen as model
from config import root

fsl.FSLCommand.set_default_output_type('NIFTI_GZ')

In [11]:
def make_contrasts(contrasts, names, con_type='T', show_output=True):
    """Make contrasts as read into FSL"""

    contrasts_fmt = []
    for contrast in contrasts:

        # create contrast title
        title_left, title_right = [], []
        for k, v in contrast.items():
            title_left += [k] if v > 0 else []
            title_right += [k] if v < 0 else []
        title = ', '.join(title_left) + ' > ' + ', '.join(title_right)

        weights_per_regr = []
        for name in names:
            weight = contrast[name] if name in contrast.keys() else 0
            weights_per_regr.append(weight)

        contrasts_fmt.append((title, con_type, names, weights_per_regr))

    if show_output:
        for con in contrasts_fmt:
            print('='*20)
            print(con[0])
            print('-'*20)
            print(con[1])
            for reg, weight in zip(con[2], con[3]):
                print(reg, '\t', weight)
            print('='*20)
            print('\n')

    return contrasts_fmt

# Prepare

In [12]:
# pathnames
base_dir = os.path.join(root, "data")
output_dir = os.path.join(root, "data", "output")
working_dir = os.path.join(root, "data", "working_dir")
mni_standard_path = os.path.join(root, "data", "in_analysis", "standard", "nii/misc/MNI152_T1_1mm_brain.nii.gz")

In [24]:
# Params
glm_prefix = 'pop_decay_'
TR = 2.1
filter_cutoff = 60

# Lists
sub_list = ['sub_%03d' % i for i in range(1, 7)]

## Input and output nodes

In [17]:
# iterate over subjects
infosource = pe.Node(
    util.IdentityInterface(
        fields=['sub_id']),
    name='infosource')

# Define iterable attributes
infosource.iterables = [('sub_id', sub_list)]

# File templates for different subjects and sessions
templates = {
    "runs"     : "output/highpass/ses_*{sub_id}/_hp_filter*/run_*_st_mcf_warp_bet_smooth_hpf.nii.gz",
    "behavior" : "search/regs/{sub_id}/ses_*/run_*.txt"
}

# SelectFiles Node to handle session specific file templates
files = pe.Node(
    SelectFiles(templates,
                base_directory=base_dir,
                sort_filelist=True),
                name='files')

# Create datasink to store important
# files in useful, more accessable locations.
datasink = pe.Node(
    DataSink(
        base_directory=base_dir,
        container="output"),
    name='datasink')

# Remove unwanted lengthy strings from filenames.
datasink.inputs.substitutions = [('_sub_id_', ''),
                                 ('_ses_id_', '')]

## Make contrasts

In [23]:
conditions = [('loc1', 'loc2', 'loc3', 'loc4'),
              ('loc_lag1', 'loc_lag2'),
              ('clr_lag1', 'clr_lag2')]

# Location contrasts
contrasts = []
for cons in conditions[:1]:
    for i, con in enumerate(cons):
        this = i
        other = list(range(len(cons)))
        other.remove(i)

        dct = {cons[this]: 1}
        dct.update({cons[i]: -1.0 / len(other) for i in other})
        contrasts.append(dct)

# Priming contrasts
for cons in conditions[1:]:
    contrasts.append({con: 1.0/len(cons) for con in cons})
    contrasts += [{con: 1.0} for con in cons]

# Stimulus onset baseline
contrasts.append({con:.25 for con in conditions[0]})

conditions = [loc for con in conditions for loc in con]

# Create contrasts for FSL
contrasts_large = make_contrasts(contrasts, conditions)

## Setup GLM

In [ ]:
glm_function = pe.MapNode(
    util.Function(
        input_names=['in_file'],
        output_names=['subject_info'],
        function=pop_decay),
    iterfield=['in_file'],
    name=glm_prefix+'glm_function')

# Specify model
specifymodel = pe.Node(
    model.SpecifyModel(
        high_pass_filter=filter_cutoff,
        time_repetition=TR,
        intput_units="secs",
    ), 
    name=glm_prefix+'specifymodel'
)

# Level 1 design
level1design = pe.Node(
    fsl.Level1Design(
        bases={"dgamma" : {"derivs": True}},
        interscan_interval=TR,
        model_serial_correlations=True, # Prewhitening
        contrasts=contrasts_large,
    ), 
    name=glm_prefix+'level1design',
    overwrite=False
)

# model
featmodel = pe.MapNode(
    interface=fsl.FEATModel(),
    name=glm_prefix+'featmodel_ses1',
    iterfield=['fsf_file', 'ev_files']
)

# FILMGLS
filmgls = pe.MapNode(
    interface=fsl.FILMGLS(
        threshold=10
    ),
    iterfield=['design_file', 'in_file', 'tcon_file'],
    name=glm_prefix+'filmgls',
    overwrite=True)

In [ ]:
# %% connect nodes
modelfit = pe.Workflow(name=glm_prefix+'modelfit')
modelfit.connect(files, 'behavior', glm_function, 'in_file')
modelfit.connect(files, 'params_clr', glm_function, 'params_clr')
modelfit.connect(files, 'params_loc', glm_function, 'params_loc')
modelfit.connect(glm_function, 'subject_info', specifymodel, 'subject_info')
modelfit.connect(specifymodel, 'session_info', level1design, 'session_info')
modelfit.connect(level1design, 'fsf_files', featmodel, 'fsf_file')
modelfit.connect(level1design, 'ev_files', featmodel, 'ev_files')
modelfit.connect(featmodel, 'design_file', filmgls, 'design_file')
modelfit.connect(featmodel, 'con_file', filmgls, 'tcon_file')

# %% Level 1
firstlevel = pe.Workflow(name='firstlevel',
                         base_dir=working_dir)

# Connect coregistration output to functional runs model specification
firstlevel.connect(preproc, 'subtract_mean.out_file',
                   modelfit, glm_prefix+'specifymodel.functional_runs')

# Connect coregistration output to model estimate input
firstlevel.connect(preproc, 'subtract_mean.out_file',
                   modelfit, glm_prefix+'filmgls.in_file')

#==============================================================================
# Outputs to output folder
#==============================================================================

firstlevel.connect(modelfit, glm_prefix+'filmgls.zstats', datasink, glm_prefix+'filmgls.@zstats')
firstlevel.connect(modelfit, glm_prefix+'filmgls.param_estimates', datasink, glm_prefix+'filmgls.@param_estimates')
firstlevel.connect(modelfit, glm_prefix+'filmgls.copes', datasink, glm_prefix+'filmgls.@copes')
firstlevel.connect(modelfit, glm_prefix+'filmgls.varcopes', datasink, glm_prefix+'filmgls.@varcopes')
firstlevel.connect(modelfit, glm_prefix+'filmgls.dof_file', datasink, glm_prefix+'filmgls.@dof_file')
firstlevel.connect(modelfit, glm_prefix+'filmgls.logfile', datasink, glm_prefix+'filmgls.@logfile')
firstlevel.connect(modelfit, glm_prefix+'filmgls.residual4d', datasink, glm_prefix+'filmgls.@residual4d')
firstlevel.connect(modelfit, glm_prefix+'filmgls.sigmasquareds', datasink, glm_prefix+'filmgls.@sigmasquareds')
firstlevel.connect(modelfit, glm_prefix+'filmgls.tstats', datasink, glm_prefix+'filmgls.@tstats')

if __name__ == '__main__':
    #firstlevel.write_graph()
    firstlevel.run(plugin='MultiProc', plugin_args={'n_procs':8})